In [2]:
import pandas as pd
import numpy as np
from scipy.integrate import simpson
from findpeaks import findpeaks
import os
import matplotlib.pyplot as plt
from cmcrameri import cm

In [3]:
def split_tsv(path_to_file):
    print(path_to_file)
    def read_file_for_headers(path_to_file):
        tsv_file = open(path_to_file, "r", encoding = "ISO-8859-1").readlines()
        header_line_idxs = []
        for line_idx in range(len(tsv_file)):
            if "!NAME!" in tsv_file[line_idx]:
                header_line_idxs.append(line_idx)
        return header_line_idxs

    header_line_idxs = read_file_for_headers(path_to_file) 

    tsv_df = pd.read_csv(path_to_file, delimiter='\t', encoding = "ISO-8859-1", on_bad_lines='skip', names=['time', 'counts'])
    
    split_df = []
    for idx in range(len(header_line_idxs)):
        header_idx = header_line_idxs[idx]
        table_start = header_idx + 1
        if idx+1 != len(header_line_idxs):
            table_end = header_line_idxs[idx+1] - 1
        else:
            table_end = tsv_df.shape[0]
            
        new_df = tsv_df.iloc[table_start: table_end]
        new_df["time"] = pd.to_numeric(new_df["time"])
        new_df = new_df.reset_index()
        name = tsv_df.iloc[header_idx].values[0]
        name = name.replace("!NAME!","")
        name= name.split('!CHROMNAME!', 1)[0]
        split_df.append({'name': name,'df': new_df})
    return split_df[:-2]

def return_peaks(df):
    fp = findpeaks(method='peakdetect',lookahead=5, interpolate=5)
    peak_results = fp.fit(df['counts'])
    peaks_df = peak_results['df']

    return peaks_df

def return_noise_height(df, peaks_df, noise_start=0, noise_end=30):
    times = df['time']
    time_closest_to_noise_start = times[times >= noise_start].min()
    time_closest_to_noise_end = times[times <= noise_end].max()
    time_closest_to_noise_start_idx = times[times == time_closest_to_noise_start].index[0]
    time_closest_to_noise_end_idx = times[times == time_closest_to_noise_end].index[0]

    peaks_df_idxs = peaks_df['x']
    peaks_df_idx_closest_to_noise_start_idx = peaks_df_idxs[peaks_df_idxs >= time_closest_to_noise_start_idx].min()
    peaks_df_idx_closest_to_noise_end_idx = peaks_df_idxs[peaks_df_idxs <= time_closest_to_noise_end_idx].max()

    noise_valleys = peaks_df[(peaks_df['x'] >= peaks_df_idx_closest_to_noise_start_idx) & (peaks_df['x'] <= peaks_df_idx_closest_to_noise_end_idx) & (peaks_df['valley'] == True)]['y']
    noise_peaks = peaks_df[(peaks_df['x'] >= peaks_df_idx_closest_to_noise_start_idx) & (peaks_df['x'] <= peaks_df_idx_closest_to_noise_end_idx) & (peaks_df['peak'] == True)]['y']

    noise_height = noise_peaks.mean() - noise_valleys.mean()

    return noise_height

def return_noise_height2(df, peaks_df, noise_exclude1, noise_exclude2):
    times = df['time']
    print(peaks_df)
    time_closest_to_noise_exclude1 = times[times >= noise_exclude1].min()
    time_closest_to_noise_exclude2 = times[times <= noise_exclude2].max()
    time_closest_to_noise_exclude1_idx = times[times == time_closest_to_noise_exclude1].index[0]
    time_closest_to_noise_exclude2_idx = times[times == time_closest_to_noise_exclude2].index[0]

    peaks_df_idxs = peaks_df['x']
    peaks_df_idx_closest_to_noise_exclude1_idx = peaks_df_idxs[peaks_df_idxs >= time_closest_to_noise_exclude1_idx].min()
    peaks_df_idx_closest_to_noise_exclude2_idx = peaks_df_idxs[peaks_df_idxs <= time_closest_to_noise_exclude2_idx].max()

    noise_valleys = peaks_df[((peaks_df['x'] <= peaks_df_idx_closest_to_noise_exclude1_idx) | (peaks_df['x'] >= peaks_df_idx_closest_to_noise_exclude2_idx)) & (peaks_df['valley'] == True)]['y']
    noise_peaks = peaks_df[((peaks_df['x'] <= peaks_df_idx_closest_to_noise_exclude1_idx) | (peaks_df['x'] >= peaks_df_idx_closest_to_noise_exclude2_idx)) & (peaks_df['peak'] == True)]['y']

    noise_height = noise_peaks.mean() - noise_valleys.mean()

    return noise_height

def mainpeak(df, peaks_df):
    try:
        maxpeak = peaks_df[peaks_df['y'] == peaks_df['y'].max()]
        maxpeak_idx = maxpeak.index[0]
        valley_idxs = peaks_df[peaks_df['valley'] == True].index.to_list()
        start_idx = np.asarray(valley_idxs)[np.asarray(valley_idxs) < maxpeak_idx].max()
        end_idx = np.asarray(valley_idxs)[np.asarray(valley_idxs) > maxpeak_idx].min()
        
        if len(maxpeak) == 1: 
            main_peak_time = df['time'].iloc[maxpeak['x'].item()]
            main_peak_count = maxpeak['y'].item()
            start_main_peak = df['time'].iloc[peaks_df.iloc[start_idx]['x'].item()]
            end_main_peak = df['time'].iloc[peaks_df.iloc[end_idx]['x'].item()]
            return {'main_peak_count': main_peak_count, 'main_peak_meantime': main_peak_time, 'main_peak_start':start_main_peak, 'main_peak_end':end_main_peak}
        if len(maxpeak) > 1: 
            maxpeak = maxpeak.iloc[0]
            print('Warning more than two max peaks - taking the first')
            main_peak_time = df['time'].iloc[maxpeak['x'].item()]
            main_peak_count = maxpeak['y'].item()
            start_main_peak = df['time'].iloc[peaks_df.iloc[start_idx]['x'].item()]
            end_main_peak = df['time'].iloc[peaks_df.iloc[end_idx]['x'].item()]
            return {'main_peak_count': main_peak_count, 'main_peak_meantime': main_peak_time, 'main_peak_start':start_main_peak, 'main_peak_end':end_main_peak}
    except:
        return {'main_peak_count': np.nan, 'main_peak_meantime': np.nan, 'main_peak_start':np.nan, 'main_peak_end':np.nan}

def return_signal_to_noise(noise_height, peak_height):
    try:
        return (peak_height * 2) / noise_height
    except:
        return 'error'

def eval_signal_to_noise(main_peak_area, sn_ratio, threshold = 100):
    print(sn_ratio)
    if type(sn_ratio) == str:
        return 'error'
    if sn_ratio < threshold:
        return 0
    
    return main_peak_area

def eicarea(df):
    area = abs(simpson(df['counts']))
    return area

def mainpeak_area(df, start_limit, end_limit):
    limit_df = df[(df['time'] <= end_limit) & (df['time'] >=  start_limit)]
    limit_area = abs(simpson(limit_df['counts']))
    return limit_area

def draw_chromatogram(df, name, dir, filename):
    fig, ax = plt.subplots(figsize=(15,5))

    x_max = max(list(df['time']))
    y_max = df['counts'].max()

    ax.set_title(f'EIC for {name}')
    ax.plot(df['time'], df['counts'], linewidth=1.0, color='#6F1F70')
    ax.fill_between(df['time'], df['counts'], 0, color='#6F1F70',alpha=0.2)
    ax.set_xticks(np.linspace(0, x_max, 6))
    
    ax.set_xlim(0, x_max)
    ax.set_ylim(0, y_max+y_max*0.05)
    ax.ticklabel_format(axis='y', style='sci')
    ax.set_xlabel('time (mins)')
    ax.set_ylabel('counts (arb)')
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    
    output_dir = os.path.join(dir, '_raw_eics')
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    plt.savefig(os.path.join(output_dir, filename+".svg"), cmap=cm.romaO)
    plt.close()

def draw_chromatogram_annonated(df, eic_area, mainpeak_info, name, dir, filename):
    fig, (ax,ax2) = plt.subplots(1,2, figsize=(10,4))

    x_max = max(list(df['time']))
    y_max = df['counts'].max()

    fig.suptitle(f'EIC for {name}')
    ax.plot(df['time'], df['counts'], linewidth=1.0, color='#D28B15')

    t = (f"SN ratio too low!\nEIC area: {eic_area} arbs\n"
         f"Main peak\n    Height: {mainpeak_info['main_peak_count']} counts\n    Apex time: {mainpeak_info['main_peak_meantime']} secs\n    Area: {mainpeak_info['main_peak_area_eval']} arbs\n    SN ratio:{mainpeak_info['sn_ratio']}")
    if mainpeak_info['sn_ratio'] > 100:
        ax.plot(mainpeak_info['main_peak_meantime'], mainpeak_info['main_peak_count'], "x")
        ax.axvline(x=mainpeak_info['main_peak_start'], color = 'gray', linestyle = '--', linewidth=1)
        ax.axvline(x=mainpeak_info['main_peak_end'], color = 'gray', linestyle = '--', linewidth=1)
    
        df_sliced = df[(df['time'] >= mainpeak_info['main_peak_start'])&(df['time'] <= mainpeak_info['main_peak_end'])]
        ax.fill_between(df_sliced['time'], df_sliced['counts'], 0, color='#D28B15',alpha=0.2)
        t = (f"EIC area: {eic_area} arbs\n"
         f"Main peak\n    Height: {mainpeak_info['main_peak_count']} counts\n    Apex time: {mainpeak_info['main_peak_meantime']} secs\n    Start time (1st grey line): {mainpeak_info['main_peak_start']} secs \n    End time (2nd grey line): {mainpeak_info['main_peak_end']} secs\n    Area: {mainpeak_info['main_peak_area_eval']} arbs\n   SN ratio:{mainpeak_info['sn_ratio']}")
    
        
        
    ax.set_xticks(np.linspace(0, x_max, 6))
    
    ax.set_xlim(0, x_max)
    ax.set_ylim(0, y_max+y_max*0.05)
    ax.ticklabel_format(axis='y', style='sci')
    ax.set_xlabel('time (mins)')
    ax.set_ylabel('counts (arb)')
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)


    ax2.spines['right'].set_visible(False)
    ax2.spines['top'].set_visible(False)
    ax2.spines['bottom'].set_visible(False)
    ax2.spines['left'].set_visible(False)
    ax2.get_xaxis().set_visible(False)
    ax2.get_yaxis().set_visible(False)
    
    ax2.text(0.05, 0.95, t, horizontalalignment='left', verticalalignment='top', transform = ax2.transAxes)
    

    output_dir = os.path.join(dir, '_annotated_eics')
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    plt.savefig(os.path.join(output_dir, filename+".svg"), cmap=cm.romaO)
    plt.close()

In [15]:
def generate_eicdata_tsv(path_to_file):
    file_name = os.path.splitext(os.path.basename(path_to_file))[0]
    file_dir = os.path.dirname(path_to_file)
    tsv_list = split_tsv(path_to_file)

    name_areas = []
    for df_dict in tsv_list:
        df = df_dict['df']
        if 'TIC' in df_dict['name'] or 'BPC' in df_dict['name']:
            eic_area = eicarea(df)
            name_areas.append({'file_name': file_name,
                               'strain_name': " ".join(file_name.split("_", 2)[:2]),
                               'repeat': file_name.split("_")[2],
                               'eic_name': df_dict['name'],
                               'eic_area': eic_area, 
                               'main_peak_count': np.nan, 
                               'main_peak_meantime': np.nan, 
                               'main_peak_start': np.nan, 
                               'main_peak_end': np.nan,
                               'main_peak_area': np.nan,
                               'noise_height': np.nan,
                               'main_peak_snratio': np.nan
                            })
            draw_chromatogram(df, file_name+df_dict['name'].replace("∆", "del"), file_dir, file_name+df_dict['name'].replace("∆", "del"))
        else:
            peaks_df = return_peaks(df)
            mainpeak_info = mainpeak(df, peaks_df)
            if mainpeak_info['main_peak_count'] != mainpeak_info['main_peak_count']:
                mainpeak_info['main_peak_area'] = np.nan
                noise_height = np.nan
                mainpeak_info['sn_ratio'] = np.nan
                mainpeak_info['main_peak_area_eval'] = np.nan
                eic_area = eicarea(df)
            else:
                print(mainpeak_info)
                mainpeak_info['main_peak_area'] = mainpeak_area(df, mainpeak_info['main_peak_start'], mainpeak_info['main_peak_end'])
                eic_area = eicarea(df)
                noise_height = return_noise_height2(df, peaks_df, mainpeak_info['main_peak_start'], mainpeak_info['main_peak_end'])
                mainpeak_info['sn_ratio'] = return_signal_to_noise(noise_height, mainpeak_info['main_peak_count'])
                mainpeak_info['main_peak_area_eval'] = eval_signal_to_noise(mainpeak_info['main_peak_area'], mainpeak_info['sn_ratio'])
                
            name_areas.append({'file_name': file_name,
                               'strain_name': " ".join(file_name.split("_", 2)[:2]),
                               'repeat': file_name.split("_")[2],
                               'eic_name': df_dict['name'],
                               'eic_area': eic_area, 
                               'main_peak_count': mainpeak_info['main_peak_count'], 
                               'main_peak_meantime': mainpeak_info['main_peak_meantime'], 
                               'main_peak_start': mainpeak_info['main_peak_start'], 
                               'main_peak_end': mainpeak_info['main_peak_end'],
                               'main_peak_area': mainpeak_info['main_peak_area'],
                               'noise_height': noise_height,
                               'main_peak_snratio': mainpeak_info['sn_ratio'],
                               'main_peak_area_eval': mainpeak_info['main_peak_area_eval']
                            })
            draw_chromatogram(df, file_name+"_"+df_dict['name'].replace("/", "_"), file_dir, file_name+"_"+df_dict['name'].replace("/", "_"))
            try:
                draw_chromatogram_annonated(df, eic_area, mainpeak_info, file_name+"_"+df_dict['name'].replace("/", "_"), file_dir, "ANNOTATED_"+file_name+"_"+df_dict['name'].replace("/", "_"))
            except:
                pass
            

    eicarea_tsv = pd.DataFrame(name_areas)
    
    return eicarea_tsv


In [16]:
def eicarea_tsv_fromdir(path_to_dir):
    df_list = []
    file_dir = os.path.dirname(path_to_dir)
    for item in os.listdir(file_dir):
        item_path = file_dir + "/" + item
        if os.path.isfile(item_path) and item_path.endswith('.tsv'):
            df_list.append(generate_eicdata_tsv(item_path))
            

    concat_df = pd.concat(df_list)
    concat_df.to_csv(os.path.join(path_to_dir, 'eic.csv'), index=False)

In [17]:
eicarea_tsv_fromdir("/Volumes/MICHAEL E-H 1/BIFFAMYCIN/Biffamycin Media Screen/17Sep_Reanalysis/1/")
eicarea_tsv_fromdir("/Volumes/MICHAEL E-H 1/BIFFAMYCIN/Biffamycin Media Screen/17Sep_Reanalysis/2/")
eicarea_tsv_fromdir("/Volumes/MICHAEL E-H 1/BIFFAMYCIN/Biffamycin Media Screen/17Sep_Reanalysis/3/")
eicarea_tsv_fromdir("/Volumes/MICHAEL E-H 1/BIFFAMYCIN/Biffamycin Media Screen/17Sep_Reanalysis/4/")

/Volumes/MICHAEL E-H 1/BIFFAMYCIN/Biffamycin Media Screen/17Sep_Reanalysis/1/EICs_Q5All_ISP2_1_1-1-10_1_1298.d.tsv
[findpeaks] >Finding peaks in 1d-vector using [peakdetect] method..
[findpeaks] >Interpolating 1d-vector by factor 5
{'main_peak_count': 3027.0, 'main_peak_meantime': 262.87, 'main_peak_start': 260.317, 'main_peak_end': 271.915}
       y    x   labx  valley   peak
0    0.0    0    1.0    True  False
1    0.0    1    1.0   False  False
2    0.0    2    2.0    True  False
3    0.0    3    2.0   False  False
4    0.0    4    3.0    True  False
..   ...  ...    ...     ...    ...
463  0.0  463  166.0   False  False
464  0.0  464  167.0   False  False
465  0.0  465  167.0   False  False
466  0.0  466  168.0   False  False
467  0.0  467  168.0   False  False

[468 rows x 5 columns]
167.6427829698858
[findpeaks] >Finding peaks in 1d-vector using [peakdetect] method..
[findpeaks] >Interpolating 1d-vector by factor 5
{'main_peak_count': 25278.0, 'main_peak_meantime': 268.544, 'main

In [42]:
eicarea_tsv_fromdir("/Volumes/MICHAEL E-H 1/BIFFAMYCIN/ORDER OF EVENTS/8September/1/eics/")
eicarea_tsv_fromdir("/Volumes/MICHAEL E-H 1/BIFFAMYCIN/ORDER OF EVENTS/8September/2/eics/")

/Volumes/MICHAEL E-H 1/BIFFAMYCIN/ORDER OF EVENTS/8September/1/eics/EICs_M1152_6HE_FIJK_1_3-2-15_1_3874.d.tsv
[findpeaks] >Finding peaks in 1d-vector using [peakdetect] method..
[findpeaks] >Interpolating 1d-vector by factor 5
{'main_peak_count': 1709.0, 'main_peak_meantime': 148.337, 'main_peak_start': 146.243, 'main_peak_end': 152.769}
         y     x   labx  valley   peak
0      0.0     0    1.0    True  False
1      0.0     1    1.0    True  False
2     34.0     2    1.0   False   True
3      0.0     3    1.0    True  False
4      0.0     4    2.0   False   True
...    ...   ...    ...     ...    ...
1130   0.0  1130  499.0    True  False
1131   0.0  1131  500.0   False  False
1132   2.0  1132  500.0   False  False
1133   1.0  1133  500.0   False  False
1134   1.0  1134  500.0   False  False

[1135 rows x 5 columns]
173.81548490700973
[findpeaks] >Finding peaks in 1d-vector using [peakdetect] method..
[findpeaks] >Interpolating 1d-vector by factor 5
{'main_peak_count': 2194.0, 'ma